## Análisis de Simulated Annealing para una fuerza de ventas de 5 nodos

Simulated Annhealing es un algoritmo que recibe los siguientes hiperárametros de entrada: 

+ Tmax
+ Tmin 
+ steps 
+ updates 

Los resultados obtenidos pueden verse afectados al variar los valores de tales hiperparámetros. Por otro lado, dependiendo del número de nodos del grafo, estos hiperparámetros también podrían afectar la ruta mínima encontrada por el algoritmo.    

El presente notebook considera la implementación de simulated annhealing para un grafo con 5 nodos. Como primero objetivo se variarán los hiperparámetros y se identificarán aquellos que den mejores resultados. Entendiéndose como mejores resultados; obtener la ruta con menor distancia. Adicionalmente, una vez seleccionados los mejores hiperparámetros, se correrá el algoritmo 10 veces con el fin de realizar un análisis sobre las rutas obtenidas. Particularmente se tiene interés en revisar variaciones en las rutas obtenidas en cada corrida.

Dentro del conjunto de datos que se tienen disponibles, existen varias fuerzas de venta que deben recorrer 5 nodos. Se decidió elegir la fuerza de venta **80993** perteneciente al estado de Nuevo León para llevar a cabo estas pruebas.

In [1]:
!pip install dynaconf
!pip install psycopg2-binary
!pip install simanneal

In [2]:
# Librerías

import pandas as pd
import sys
sys.path.append('../')
%load_ext autoreload
%autoreload 2

from src import Utileria as ut
from src.models import particle_swarm as ps
from src.models import simulated_annealing as sa

### 1. Búsqueda de mejores hiperparámetros

**1.1 Definición de datos de entrada**
+ Grafo completo de los puntos que debe vistar la fuerza de ventas
+ Hiperámetros con los que correrá el algoritmo

In [3]:
# Se obtiene el dataframe que contiene el grafo de la fuerza de venta a evaluar:
str_Query = 'select id_origen, id_destino, distancia from trabajo.grafos where id_fza_ventas={};'

# En el query se especifica el id_fza_venta del cual se quiere obtener su grafo
df_Grafo = ut.get_data(str_Query.format(80993))
df_Grafo

Selecting rows from table using cursor.fetchall
PostgreSQL connection is closed


,id_origen,id_destino,distancia
0,11037,1001402004,0.267770992210818
1,11037,1006681965,0.3250713086280114
2,11037,1020053072,2.3599263518563016
3,11037,1020235635,1.2662382942527688
4,11037,1020402992,0.07674597387253217
5,1001402004,1006681965,0.4982193543875845
6,1001402004,1020053072,2.611292644038669
7,1001402004,1020235635,1.5321611628202725
8,1001402004,1020402992,0.23296305988637042
9,1006681965,1020053072,2.141257356651251


In [22]:
# Se crea el diccionario de hiper-parámetros que se evaluarán

#Default parameters
#Tmax = 25000.0  # Max (starting) temperature
#Tmin = 2.5      # Min (ending) temperature
#steps = 50000   # Number of iterations
#updates = 100   # Number of updates (by default an update prints to stdout)

dict_Hiper_SA = {'Tmax': {10000, 25000},
              'Tmin': {1,2.5,5},
              'steps': {500,5000},
              'updates': {10,50,100}
              }



**1.2 Gridsearch**

Dentro de la clase Utileria fue definido un método llamado *GridSearch*, el cual recibe como parámetros el grafo de una fuerza de ventas fijo, un diccionario de parámetros, el algoritmo a evaluar y el número de iteraciones que se correrá por cada combinación de hiperámetros. Este método evalúa el algoritmo con todas las combinaciones que se pueden generar a partir del diccionario de parámetros. 

En este caso se considerarán 3 valores de *updates*, 2 valores del *steps*, 2 valores de *Tmax* y 2 de *Tmin* dando lugar a un total de 36 combinaciones. 

Cada combinación de hiperarámetros se correrá 10 veces y como resultado se obtendrá una tabla indicando los Hiperámetros utilizados, las distancias mínima y máxima obtenidas dentro de las 10 corridas; y el número de corridas en que se repitió tal distancia mínima.

In [ ]:
%%time

# Se corre el GridSearch para el grafo y los hiperparámetros previamente definidos

df_Resultado = ut.GridSearch(df_Grafo, sa.SimulatedAnnealing, dict_Hiper_SA, 100)

 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.17000          4.00    68.60%     0.00%     0:00:01    -1:59:59 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.17000          4.00    68.26%     0.00%     0:00:07     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.26000          4.00    67.35%     0.00%     0:00:01    -1:59:59 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.26000          4.00    68.48%     0.00%     0:00:08     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.26000          4.00    67.70%     0.00%     0:00:01    -1:59:59 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.26000          4.00    68.67%     0.00%     0:00:07     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.26000          4.00    70.70%     0.00%     0:00:01    -1:59:59 Temperature 

In [10]:
# Se muestra el dataframe con los resultados obtenidos de la corrida del GridSearch
pd.options.display.max_colwidth = 100
df_Resultado

,HiperParámetros,Distancia mínima (km),Distancia máxima (km),Frec. rel. dist. min.
0,"{'Tmax': 10000, 'Tmin': 1.0, 'steps': 5000, 'updates': 100}",4,4,10/10
1,"{'Tmax': 10000, 'Tmin': 1.0, 'steps': 5000, 'updates': 10}",4,4,10/10
2,"{'Tmax': 10000, 'Tmin': 1.0, 'steps': 5000, 'updates': 50}",4,4,10/10
3,"{'Tmax': 10000, 'Tmin': 1.0, 'steps': 500, 'updates': 100}",4,4,10/10
4,"{'Tmax': 10000, 'Tmin': 1.0, 'steps': 500, 'updates': 10}",4,4,10/10
5,"{'Tmax': 10000, 'Tmin': 1.0, 'steps': 500, 'updates': 50}",4,4,10/10
6,"{'Tmax': 10000, 'Tmin': 2.5, 'steps': 5000, 'updates': 100}",4,4,10/10
7,"{'Tmax': 10000, 'Tmin': 2.5, 'steps': 5000, 'updates': 10}",4,4,10/10
8,"{'Tmax': 10000, 'Tmin': 2.5, 'steps': 5000, 'updates': 50}",4,4,10/10
9,"{'Tmax': 10000, 'Tmin': 2.5, 'steps': 500, 'updates': 100}",4,4,10/10


**1.3 Análisis y Resultados**

En primer lugar es importante mencionar las motivaciones de ciertos de los hiperparámetros elegidos. 

+ Con respecto al número de iteraciones,se escogieron $10$ iteraciones para poder ver la variación entre iteraciones y la estabilidad del algoritmo. El número fue suficiente ya que en ninguna combinación de parámetros hubo variación en la distancia minima. 

+ Sobre el resto de los parámetros, se escogieron valores alrededor e incluyendo los parámetros por omisión del algoritmo. 

Para poder interpretar los resultados mostrados en el dataframe anterior es conveniente recordar que se realizaron 36 pruebas, es decir, se tuvieron 36 combinaciones de hiperparámetros; y cada una de esas combinaciones se corrió 10 veces, dando un total de 360 corridas. En las 10 corridas de cada prueba se registró la distancia mínima obtenida, la distancia máxima y la frecuencia relativa de la distancia mínima. A continuación se enlistan observaciones importantes derivadas de estas pruebas:

+ La distancia mínima y máxima obtenida: $4 km$ es la misma para las 36 combinaciones. Esto demuestra que apesar de utilizar los mismos hiperparámetros, el algoritmo da los mismos resultados. 
 
+ Estos resultados pueden ser consecuencia de haber utilizando los parámetros por default, por eso se hizo el siguiente experimento: correr el algoritmo 10 veces solo con 1 paso, updates con valor 2. Tmin con valor 1 y Tmax con valor 2. Fue interesante que la distancia minima y máxima de nuevo fue $4 km$. Puede que la estabilidad sea resultado del número pequeño de nodos. 


In [11]:
%%time

# Se corre el GridSearch para el grafo y los hiperparámetros previamente definidos

dict_Hiper_SA = {'Tmax': {2},
              'Tmin': {1},
              'steps': {1},
              'updates': {2}
              }


df_Resultado = ut.GridSearch(df_Grafo, sa.SimulatedAnnealing, dict_Hiper_SA, 10)

 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.26000          4.00    67.25%     0.00%     0:00:01    -1:59:59 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.26000          4.00    74.68%     3.37%     0:00:06     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.17000          4.00    66.70%     0.00%     0:00:01    -1:59:59 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.17000          4.00    72.35%     2.24%     0:00:06     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.26000          4.00    66.50%     0.00%     0:00:01    -1:59:59 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.26000          4.00    74.60%     3.34%     0:00:07     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.17000          4.00    68.00%     0.00%     0:00:01    -1:59:59 Temperature 

CPU times: user 1min 15s, sys: 281 ms, total: 1min 15s
Wall time: 1min 16s


In [12]:
pd.options.display.max_colwidth = 100
df_Resultado

,HiperParámetros,Distancia mínima (km),Distancia máxima (km),Frec. rel. dist. min.
0,"{'Tmax': 2, 'Tmin': 1, 'steps': 1, 'updates': 2}",4,4,10/10


### 2. GridSearch con parametros simples

In [17]:
rutas = pd.DataFrame(index=range(3),columns=['km', 'Ruta'])


dict_Hiper_SA = {'Tmax': 2,
              'Tmin': 1,
              'steps': 1,
              'updates': 2
              }


In [18]:
rutas

,km,Ruta
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN


In [19]:
for corrida in range(3):
    SA = sa.SimulatedAnnealing(df_Grafo,dict_Hiper_SA)
    SA.Ejecutar()
    
    min_distancia = round(SA.nbr_MejorCosto,3)
    mejor_ruta = SA.lst_MejorCamino
    
    rutas.km[corrida] = min_distancia
    rutas.Ruta[corrida] = mejor_ruta
    
    

 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.26000          4.00    68.10%     0.00%     0:00:01    -1:59:59 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.26000          4.00    73.41%     2.77%     0:00:06     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.26000          4.00    66.55%     0.00%     0:00:01    -1:59:59 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.26000          4.00    74.72%     3.34%     0:00:06     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.26000          4.00    67.40%     0.00%     0:00:01    -1:59:59 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.26000          4.00    73.55%     2.84%     0:00:06     0:00:00

In [20]:
rutas

,km,Ruta
0,4,"[1006681965, 1020235635, 1020053072, 1001402004, 11037]"
1,4,"[1001402004, 1006681965, 11037, 1020053072, 1020235635]"
2,4,"[1020235635, 1020053072, 11037, 1006681965, 1001402004]"


### Conclusiones:
+ Los resultados del algoritmo pueden variar en cada corrida, aún cuando se mantengan fijos los hiperparámetros.
+ Este problema tiene múltiples minimos y regresa diferentes rutas pero con el mismo mínimo. 


In [21]:
def convert(ruta): 
    s = [str(i) for i in ruta] 
    ruta_c = "-".join(s)
    return(ruta_c) 